##BB84 Implementation


In [107]:
from qiskit_aer import AerSimulator,Aer
from qiskit import QuantumCircuit,QuantumRegister,ClassicalRegister,transpile
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Statevector
from numpy import random

In [108]:
def random_bit_generator(n):
    return [str(random.randint(0,2)) for _ in range(n)]

In [109]:
def random_basis(n):
    return [random.choice(["X","Z"]) for _ in range(n)]
    

In [110]:
#Step 1: Generating basis and bits
n=12
Alice_bits=random_bit_generator(n)
Alice_base=random_basis(n)
qc=QuantumCircuit(n,n)

#Eve's Eavedropping
Eve_base=random_basis(n)


#Bobs measurement basis
Bob_base=random_basis(n)
#Step 2: Encoding bits
for i in range(len(Alice_bits)):
    if Alice_bits[i]=="1":
        qc.x(i)
    if Alice_base[i]=="X":
         qc.h(i)
qc.barrier()

#Creates copy of circuit
Eveqc=QuantumCircuit(n,n)
Eveqc.data=qc[:]
#Eve's measurement
for i in range(len(Eve_base)):
    if Eve_base[i]=="X":
        Eveqc.h(i)
    Eveqc.measure(i,i)



#Runs the circuit
sim=AerSimulator()
compliled=transpile(Eveqc,sim)

result=sim.run(compliled,shots=1).result()
stats=result.get_counts()
Eve_bits=list(stats.most_frequent())
Eve_bits=list(reversed(Eve_bits))


#New Circuit for Bob
qc2=QuantumCircuit(n,n)
#Rencoding bits
for i in range(len(Eve_bits)):
    if Eve_bits[i]=="1":
        qc2.x(i)
    if Eve_base[i]=="X":
         qc2.h(i)
qc2.barrier()





#Step 3:Bob measurement
for i in range(len(Bob_base)):
    if Bob_base[i]=="X":
        qc2.h(i)
    qc2.measure(i,i)
#qc.draw("mpl")


In [111]:
sim=AerSimulator()
compliled=transpile(qc2,sim)

result=sim.run(compliled,shots=1).result()
stats=result.get_counts()
measured_bits=list(stats.most_frequent())
measured_bits=list(reversed(measured_bits))




In [112]:
matchingE=[i for i in range(n) if Alice_base[i]==Eve_base[i]]
Eve_key=[Eve_bits[i] for i in matchingE]
matching=[i for i in range(n) if Alice_base[i]==Bob_base[i]]
Alice_key=[Alice_bits[i] for i in matching]
Bob_key=[measured_bits[i] for i in matching]

print(matching)
print(matchingE)
print(Alice_key)
print(Bob_key)
print(Eve_key)


[1, 2, 3, 4, 8, 9, 10, 11]
[0, 3, 4, 7, 8, 9, 10, 11]
['1', '0', '0', '1', '0', '0', '1', '1']
['1', '1', '0', '1', '0', '0', '1', '1']
['0', '0', '1', '0', '0', '0', '1', '1']


In [113]:
#To check running circuits multiple times

def run_bb84_with_eve(n=12):
    # Step 1: Generate random bits and bases
    Alice_bits = random_bit_generator(n)
    Alice_base = random_basis(n)
    Eve_base = random_basis(n)
    Bob_base = random_basis(n)

    # Alice encodes her bits
    qc = QuantumCircuit(n, n)
    for i in range(n):
        if Alice_bits[i] == "1":
            qc.x(i)
        if Alice_base[i] == "X":
            qc.h(i)
    qc.barrier()

    # Eve intercepts and measures
    Eveqc = QuantumCircuit(n, n)
    Eveqc.data = qc[:]
    for i in range(n):
        if Eve_base[i] == "X":
            Eveqc.h(i)
        Eveqc.measure(i, i)
    sim = AerSimulator()
    compliled = transpile(Eveqc, sim)
    result = sim.run(compliled, shots=1).result()
    stats = result.get_counts()
    Eve_bits = list(stats.most_frequent())
    Eve_bits = list(reversed(Eve_bits))

    # Bob receives and measures
    qc2 = QuantumCircuit(n, n)
    for i in range(n):
        if Eve_bits[i] == "1":
            qc2.x(i)
        if Eve_base[i] == "X":
            qc2.h(i)
    qc2.barrier()
    for i in range(n):
        if Bob_base[i] == "X":
            qc2.h(i)
        qc2.measure(i, i)
    compliled = transpile(qc2, sim)
    result = sim.run(compliled, shots=1).result()
    stats = result.get_counts()
    measured_bits = list(stats.most_frequent())
    measured_bits = list(reversed(measured_bits))

    # Key sifting
    matchingE = [i for i in range(n) if Alice_base[i] == Eve_base[i]]
    Eve_key = [Eve_bits[i] for i in matchingE]
    matching = [i for i in range(n) if Alice_base[i] == Bob_base[i]]
    Alice_key = [Alice_bits[i] for i in matching]
    Bob_key = [measured_bits[i] for i in matching]

    return {
        "Alice_bits": Alice_bits,
        "Alice_base": Alice_base,
        "Eve_base": Eve_base,
        "Bob_base": Bob_base,
        "Eve_bits": Eve_bits,
        "measured_bits": measured_bits,
        "matching": matching,
        "matchingE": matchingE,
        "Alice_key": Alice_key,
        "Bob_key": Bob_key,
        "Eve_key": Eve_key
    }

In [114]:
edata=[]
for _ in range(10):
    ecount=0
    result = run_bb84_with_eve(12)
    resA=result["Alice_key"]
    resB=result["Bob_key"]
    resE=result["Eve_key"]
    '''
    print("Alice's key:", resA)
    print("Bob's key:", resB)
    print("Eve's key:", resE) '''
    for i in range(len(resA)):
        if resA[i] != resB[i]:
            ecount+=1
    edata.append(ecount/len(resA) * 100)  # Calculate error rate as a percentage
avgError = sum(edata) / len(edata)
print(f"Average error rate over 10 runs: {avgError:.2f}%")
print("Calculated error is more than threshold(10%), so the keys are not secure." if avgError > 10 else "Calculated error is within threshold(10%), so the keys are secure.")

Average error rate over 10 runs: 24.10%
Calculated error is more than threshold(10%), so the keys are not secure.
